In [0]:
import pymysql
import pandas as pd

In [0]:
jdbcHostname = "db4free.net" #"35.80.8.91"
jdbcDatabase = "exocube" #"KAGGLE"
jdbcPort = "3306" #3306
jdbcUsername = "sample_jenish"
jdbcPassword = "Jenu7492@patel"
table_name = "employeeinfo"#'products'

In [0]:
def get_connection(jdbcHostname, jdbcUsername, jdbcPassword, jdbcDatabase):
    try:
        connection = pymysql.connect(host=jdbcHostname,user=jdbcUsername,password=jdbcPassword,db=jdbcDatabase)
#This exception is raised for things that are not in control of the programmer. For e.g unexpected disconnect, error in memory allocation etc, selected database not exists.
    except pymysql.DatabaseError as e:
        if e.args[0] == 2003:
            print('server not found')
        elif e.args[0] == 1044:
            print('database not found')
        elif e.args[0] == 1045:
            print('username or password incorrect')
    except:
        print("Unknown error occurred")   
    else:
        return connection

In [0]:
def desc_table(schema_query, connection):
    try:
        schema_df = pd.read_sql(schema_query, connection)
    except pymysql.ProgrammingError as e:
        if e.args[0] == 1146:
            print('table doesnt exist')
        elif e.args[0] == 1064:
            print('sql query syntax error')  
    else:
        return schema_df
    finally:
        connection.close()

In [0]:
schema_query = 'describe {}'.format(table_name)
connection = get_connection(jdbcHostname, jdbcUsername, jdbcPassword, jdbcDatabase)

In [0]:
schema_df = desc_table(schema_query, connection)
schema_df

,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(255),YES,,None,
2,department,varchar(255),YES,,None,


In [0]:
schema_df['Type'] = schema_df['Type'].str.replace(r'(^.*int.*$)', 'integer',regex=True)
schema_df['Type'] = schema_df['Type'].str.replace(r'(^.*varchar.*$)', 'string',regex=True)
schema_df['Type'] = schema_df['Type'].replace({"datetime": "timestamp", "float": "double"})

schema_field = schema_df['Field']
new_schema_type = schema_df['Type']

bronze_genrate_query = 'create table if not exists bronze.'+table_name+' ('
silver_genrate_query = 'create table if not exists silver.'+table_name+' ('

df_len = len(schema_df)

for i in range(df_len):
    if i != df_len-1:
        bronze_genrate_query = bronze_genrate_query+''+schema_field[i]+' '+new_schema_type[i]+', '
        silver_genrate_query = silver_genrate_query+''+schema_field[i]+' '+new_schema_type[i]+', '
    else:
        bronze_genrate_query = bronze_genrate_query+''+schema_field[i]+' '+new_schema_type[i]+', run_id integer)'
        bronze_genrate_query = bronze_genrate_query+' PARTITIONED BY(run_id)'
        silver_genrate_query = silver_genrate_query+''+schema_field[i]+' '+new_schema_type[i]+', run_id integer)'
        
        print('Bronze :- ',bronze_genrate_query)
        print('\nSilver :- ',silver_genrate_query)

        spark.sql(bronze_genrate_query)
        spark.sql(silver_genrate_query)

Bronze :-  create table if not exists bronze.employeeinfo (id integer, name string, department string, run_id integer) PARTITIONED BY(run_id)

Silver :-  create table if not exists silver.employeeinfo (id integer, name string, department string, run_id integer)
